<a href="https://colab.research.google.com/github/vinayak2019/ml_for_molecules/blob/main/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Overview

We will use pytorch for training our neural networks. First, we will look at the overall pipeline to train and validate the model. For this, we will use ML models from already available python pacakges like ``dgl`` and ``deepchem``. 


The process of training and validating model is below --

1. Clean up dataset
2. Featurize the data
3. Split the dataset
4. Create ``DataLoader`` for the dataset splits
5. Create the ML model, define loss function and optimizer
6. ``for`` loop for epochs
    1. ``for`` loop for training batches
        1. Do a forward pass
        2. Compute the loss
        3. Do backpropogation
    2. ``for`` loop for validation batches
        1. Do a forward pass
        2. Compute the loss
    

### Installing the packages

In [ ]:
# install deepchem, dgl, rdkit and fast-ml
! pip install deepchem
! pip install dgl
! pip install dgllife
! pip install rdkit
! pip install fast_ml

### Dataset operations

We will fetch the QM9 dataset and use HOMO-LUMO gap as the target for prediction. 

In [ ]:
# import pandas library
import pandas as pd

# load the dataframe as CSV from URL. 
df = pd.read_csv("https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")

# create the dataset with smiles and gap
dataset = df[["smiles","gap"]].sample(frac=0.1)

For featurizing the SMILES, we will use ``CircularFingerprint`` from deepchem.

In [ ]:
# import depechem and rdkit
import deepchem as dc
from rdkit import Chem

# create the featurizer object
featurizer = dc.feat.CircularFingerprint(radius=2, size=100)

# apply the featurizer to dataset
dataset["fp"] = dataset["smiles"].apply(featurizer.featurize)

# the fp is an array; we will convert it to a list as required for model input 
dataset["fp"] = dataset["fp"].apply(lambda x: list(x[0]))

# just use the fp and gap part of the dataset
dataset = dataset[["fp","gap"]]

Here, we use random splitting with Fast-ML. Other splitters could also be used.

In [ ]:
# import the function to split into train-valid-test
from fast_ml.model_development import train_valid_test_split

X_train, y_train, X_valid, y_valid, \
X_test, y_test = train_valid_test_split(dataset, target = "gap", train_size=0.8,
                                        valid_size=0.1, test_size=0.1) 

Checking the dataset before procceding.

In [ ]:
X_test.head()

### Dataloader

The ``DataLoader`` helps in creating batches, shuffling data and feeding the data into to model during training. The dataloader requires the dataset in the form (X,y) where X is the input and y is the target.

The dataloader code below does this transformation. The ``collate_data`` function is need for batching the (X,y) entries before feeding the batches into the model.

In [ ]:
def collate_data(data):
  # our data is in the form of list of (X,y)
  # the map function thus maps accordingly
  X, y = map(list, zip(*data))

  # we need to stack the Xs and ys for different entries in the batch
  X = torch.stack(X, dim=0)
  y = torch.stack(y, dim=0)
  return X, y

In [ ]:
# import dataloader
import torch
from torch.utils.data import DataLoader

# create the dataloader for train dataset
# dataset should be of form (X,y) according to the collate function
# the inputs should also be converted to tensors
train_dataloader = DataLoader(
    dataset=list(zip(torch.tensor(X_train["fp"].values.tolist(), dtype=torch.float32),
                     torch.tensor(y_train.tolist(), dtype=torch.float32))),
    batch_size=64, collate_fn=collate_data)

We can look at the first entry of the train_dataloader with -

In [ ]:
train_dataloader.dataset[0]

Repeat the same for the valid_dataset and test_dataset

In [ ]:
valid_dataloader = DataLoader(
    dataset=list(zip(torch.tensor(X_valid["fp"].values.tolist(), dtype=torch.float32),
                     torch.tensor(y_valid.tolist(), dtype=torch.float32))),
    batch_size=64, collate_fn=collate_data)

test_dataloader = DataLoader(
    dataset=list(zip(torch.tensor(X_test["fp"].values.tolist(), dtype=torch.float32),
                     torch.tensor(y_test.tolist(), dtype=torch.float32))),
    batch_size=64, collate_fn=collate_data)

### Model, loss and optimizer

ML models architectures are already available in many python packages including deepchem, dgl-lifesci, chemprop, chemml.

Here, we will use a model from dgl-lifesci package to show the process of training a ML model. deepchem has a streamlined process and you cannot see what happens behind the scene. We will look at one such implementation later.

In [ ]:
# import MLP model from dgl-lifesci
from dgllife.model.model_zoo.mlp_predictor import MLPPredictor

model = MLPPredictor(in_feats=100, hidden_feats=512, n_tasks=1, dropout=0.)
model

The ``MLPPredictor`` is a simple two layer model with dropout, batchnorm and ReLU activation. This alone may not be a good model, but is inexpensive for training demonstration.

Once the model is created, we can set the loss function

In [ ]:
# loss function for regresssion is usually mean squared error
import torch

loss_func = torch.nn.MSELoss(reduce=None)

We will use the Adam optimizer for training.

In [ ]:
# adam optimier
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Model training and validation

We follow the steps in the overview

In [ ]:
epochs = 5

# loop over epochs
for epoch in range(epochs):
  print("\nStarting Epoch", epoch+1)

  model.train()
  # loop over training batches

  train_loss = []
  for batch in train_dataloader: 

    # Do a forward pass
    feats, target = batch
    predictions = model(feats)
  
    # Compute loss and gradient
    loss = (loss_func(predictions, target)).mean()
    optimizer.zero_grad()

    # Do back propogation
    loss.backward()
    optimizer.step()

    # save loss to compute average loss
    train_loss.append(loss)

  print("Training loss", torch.tensor(train_loss).mean().item())


  # loop over validation batches
  model.eval()
  valid_loss = []
  with torch.no_grad():
    for batch in valid_dataloader:
      
      # Do a forward pass
      feats, target = batch
      predictions = model(feats)
    
      # Compute loss and gradient
      loss = (loss_func(predictions, target)).mean()
      valid_loss.append(loss)
  print("Validation loss ", torch.tensor(valid_loss).mean().item())


### Testing the performance

We can get a random sample from the test dataset and look at the predicted and true value

In [ ]:
idx = 159
x_sample = X_test["fp"].iloc[idx]
y_sample = y_test.iloc[idx]

y_sample

In [ ]:
model.eval()
model(torch.tensor([x_sample], dtype=torch.float32))

Let us get prediction over the entire test dataset

In [ ]:
predicted_values = []
true_values = y_test.to_list()

model.eval()
for sample in X_test["fp"].tolist():
   prediction = model(torch.tensor([x_sample], dtype=torch.float32))
   predicted_values.append(prediction.item())

We can create a scatter plot to look at the correlation

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(true_values, predicted_values)

As we noted before, the model predictions are not good; predicts nearly constant value. We can also arrive at the conclusion from the R<sup>2</sup> score

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

print("R2 score ", r2_score(true_values,predicted_values))